In [1]:
%pylab inline
import sqlite3,copy,gc,os
import pandas as pd
import statsmodels.formula.api as sm
import numpy as np
import matplotlib.pyplot as plt 
from statsmodels.tsa.stattools import adfuller
import multiprocessing as mp
import time
import scipy

matplotlib.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


/Users/sn0wfree/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
def InitalSqliteConnection(target):

    if target != 0:
        conn = sqlite3.connect(target)
    else:
        conn = sqlite3.connect(":memory:")

    return conn

In [3]:
BG_conn = InitalSqliteConnection('Bloomberg_grouped.db')
GS_conn = InitalSqliteConnection('GoogleSearch_grouped.db')
otherfactor =pd.read_csv('/Users/sn0wfree/Documents/python_projects/data_collection/API/otherfactor/otherfactor.csv')[['date','TBill1Mo','MktRF' ,u'SMB', u'HML',u'GS_SMB', u'GS_HML',u'BG_SMB', u'BG_HML']]

col_name =['HighMinusLow','Group_0', 'Group_1', 'Group_2', 'Group_3', 'Group_4', 'Group_5', 'Group_6', 'Group_7', 'Group_8', 'Group_9']



In [4]:
def calHML(df):
    df['HighMinusLow'] =df['Group_9']-df['Group_0']
    return df
def mergeotherfactor(df,otherfactor):
    return pd.merge(df,otherfactor,on='date')
def ReturnPTandParams(GStestFF3,Model_name):
    params={}
    tvalue={}
    pvalue={}
    for key,value in GStestFF3.items():
        params.update({key :value.params})
        tvalue.update({key :value.tvalues})
        pvalue.update({key :value.pvalues})
    params,pvalue,tvalue =pd.DataFrame(params).T,pd.DataFrame(pvalue).T,pd.DataFrame(tvalue).T 
    return pd.concat([RenameColumns(params,'the (%s)Coeff of'%Model_name),RenameColumns(tvalue,'T value of'),RenameColumns(pvalue,'pvalue of the tvalue of'),RenameColumns(ReturnADJR2andFpvalue(GStestFF3),0)],axis=1)

def ReturnADJR2andFpvalue(GStestCAPM):
    adjr={}
    for group,result in GStestCAPM.items():
        adjr[group]=(result.rsquared_adj,result.f_pvalue)
    return pd.DataFrame(adjr,index=['Adj_R2','f_pvalue']).T
#dir(BGtestCAPM['Group_0'])


def CAPMbased(BGMAP_10,default_weighted='ewrt'):
    

    col_name =['HighMinusLow','Group_0', 'Group_1', 'Group_2', 'Group_3', 'Group_4', 'Group_5', 'Group_6', 'Group_7', 'Group_8', 'Group_9']

    BGtestFF3={}
    BGtestCAPM={}
    
    for group in col_name:
        #print target
        
        #locals()['BG'+default_weighted ] = pd.merge(target,otherfactor,on='date')
        #print locals()['BG'+target+r ].head()
        #locals()['GS'+target+r ] = pd.merge(BGMAP_10['df_vwrt'],otherfactor,on='date')
        CMDFF3 =[group]+['MktRF' ,'SMB', 'HML']
        CMDCAPM =[group]+['MktRF']
        testdf = BGMAP_10[CMDFF3]
        testdf.dropna()
        #GStestFF3[group] = sm.ols(formula='~'.join(CMDFF3[0:2])+'+'+"+".join(CMDFF3[2:]), data=locals()['GS'+target+r ][CMDFF3]).fit()

        #GStestCAPM[group] = sm.ols(formula='~'.join(CMDCAPM[0:2]), data=locals()['GS'+target+r ][CMDCAPM]).fit()
        BGtestFF3[group] = sm.ols(formula='~'.join(CMDFF3[0:2])+'+'+"+".join(CMDFF3[2:]), data=testdf).fit()

        BGtestCAPM[group] = sm.ols(formula='~'.join(CMDCAPM[0:2]), data=testdf).fit()

    return pd.concat([ReturnPTandParams(BGtestFF3,'FF3'),ReturnPTandParams(BGtestCAPM,'CAPM')],axis=1)
#BGMAP_10['df_vwrt']
def CalculateSuccessRate(BGMAP_10):
    BG_test_postive =np.sign(BGMAP_10['df_vwrt'].set_index('date'))>0

    BGSuccessRate =pd.DataFrame(BG_test_postive.describe().T['freq']/BG_test_postive.describe().T['count'])
    BGSuccessRate.columns=['SuccessRate']
    return pd.concat([BGMAP_10['vwrt'],BGSuccessRate],axis=1)
    #return BGSuccessRate


def RenameColumns(df,name):
    if isinstance(name ,list) or isinstance(name ,set) or isinstance(name ,tuple):
        df.columns =name
    elif isinstance(name,str):
        if name in df.columns[0]:
            pass
        else:
            df.columns=[name +col for col in df.columns]
    else:
        pass
    return df


In [14]:
BGMAP_10_test = mergeotherfactor(calHML(BGMAP_10), otherfactor)
GSMAP_10_test = mergeotherfactor(calHML(GSMAP_10), otherfactor)
BGRET_test = mergeotherfactor(calHML(BGRET), otherfactor)
col_name =['HighMinusLow','Group_0', 'Group_1', 'Group_2', 'Group_3', 'Group_4', 'Group_5', 'Group_6', 'Group_7', 'Group_8', 'Group_9']
same =['MktRF' ,'SMB', 'HML']
col =['Group_0']+same
CAPMbased(BGMAP_10_test)
#CAPMbased(GSMAP_10_test)
BGRET_test.mean()


date            2.013180e+07
Group_0         5.756899e-04
Group_1         6.566416e-04
Group_2         6.966527e-04
Group_3         6.434319e-04
Group_4         6.333971e-04
Group_5         6.443328e-04
Group_6         8.218015e-04
Group_7         6.615248e-04
Group_8         7.886234e-04
Group_9         9.881522e-04
HighMinusLow    4.124623e-04
TBill1Mo        3.661121e-04
MktRF           5.177951e-04
SMB             1.479928e-05
HML             5.650090e-05
GS_SMB         -9.526421e-05
GS_HML          3.307450e-05
BG_SMB         -8.792826e-05
BG_HML          3.787525e-05
dtype: float64

In [15]:
GSMAP_10.mean()

date            2.010073e+07
Group_0         1.033413e-02
Group_1         1.033537e-02
Group_2         1.029719e-02
Group_3         1.031603e-02
Group_4         1.018671e-02
Group_5         1.023915e-02
Group_6         1.016929e-02
Group_7         1.010254e-02
Group_8         1.001881e-02
Group_9         1.017895e-02
HighMinusLow   -1.551827e-04
dtype: float64

In [6]:
BGRET=pd.read_sql_query('select * from RETewrt',BG_conn)
GSRET=pd.read_sql_query('select * from RETewrt',GS_conn)
BGMA_10=pd.read_sql_query('select * from MAP_10ewrt',BG_conn)
GSMA_10=pd.read_sql_query('select * from MAP_10ewrt',GS_conn)
BGMAP_10=pd.read_sql_query('select * from realMAP_10ewrt',BG_conn)
GSMAP_10=pd.read_sql_query('select * from realMAP_10ewrt',GS_conn)

def allols(ls,conn,sy):
    for l in ls:
        locals()['%s%s'%(sy,l)] =pd.read_sql_query('select * from %s'%(l+'ewrt'),conn)
        CAPMbased(mergeotherfactor(calHML(locals()['%s%s'%(sy,l)]), otherfactor)).to_csv('%s%s_test.csv'%(sy,l))

ls =['realMAP_10','realMAP_20','realMAP_50','realMAP_100','realMAP_200']        
allols(ls,BG_conn,'BG')    

In [7]:
GSMAP_10_vwrt=pd.read_sql_query('select * from realMAP_10vwrt',GS_conn)

In [8]:
CAPMbased((mergeotherfactor(calHML(GSMAP_10_vwrt),otherfactor)))

,the (FF3)Coeff ofIntercept,the (FF3)Coeff ofMktRF,the (FF3)Coeff ofSMB,the (FF3)Coeff ofHML,T value ofIntercept,T value ofMktRF,T value ofSMB,T value ofHML,pvalue of the tvalue ofIntercept,pvalue of the tvalue ofMktRF,...,Adj_R2,f_pvalue,the (CAPM)Coeff ofIntercept,the (CAPM)Coeff ofMktRF,T value ofIntercept,T value ofMktRF,pvalue of the tvalue ofIntercept,pvalue of the tvalue ofMktRF,Adj_R2,f_pvalue
Group_0,0.009178,-0.720666,-0.048954,0.179447,54.290441,-44.941089,-1.580233,6.133021,0.000000,0.000000e+00,...,0.424283,0.000000e+00,0.009181,-0.688343,53.946532,-48.097158,0.00000,0.000000,0.416462,0.000000
Group_1,0.009146,-0.719994,-0.097043,0.126811,52.168095,-43.292724,-3.020454,4.178987,0.000000,1.976263e-323,...,0.415956,0.000000e+00,0.009148,-0.703555,51.930464,-47.494285,0.00000,0.000000,0.410342,0.000000
Group_2,0.009149,-0.712986,-0.089648,0.160455,52.735973,-43.323188,-2.819679,5.343443,0.000000,9.881313e-324,...,0.412040,0.000000e+00,0.009152,-0.688788,52.409180,-46.905828,0.00000,0.000000,0.404319,0.000000
Group_3,0.008956,-0.710217,-0.064577,0.188098,50.952390,-42.595648,-2.004802,6.182797,0.000000,3.668270e-315,...,0.398736,0.000000e+00,0.008960,-0.677696,50.604309,-45.518739,0.00000,0.000000,0.389940,0.000000
Group_4,0.008741,-0.713116,0.011120,0.177956,50.475755,-43.409193,0.350388,5.936919,0.000000,0.000000e+00,...,0.402335,0.000000e+00,0.008745,-0.674944,50.236312,-46.105748,0.00000,0.000000,0.396056,0.000000
Group_5,0.008798,-0.714067,0.021329,0.145287,51.356919,-43.941028,0.679403,4.899871,0.000000,0.000000e+00,...,0.410920,0.000000e+00,0.008802,-0.681647,51.202741,-47.157174,0.00000,0.000000,0.406897,0.000000
Group_6,0.008668,-0.689295,0.012039,0.157799,51.206535,-42.925150,0.388083,5.385650,0.000000,4.487895e-319,...,0.398351,0.000000e+00,0.008672,-0.655224,51.011859,-45.834532,0.00000,0.000000,0.393236,0.000000
Group_7,0.008659,-0.688820,0.046451,0.147231,50.831119,-42.629623,1.488068,4.993810,0.000000,1.450229e-315,...,0.393614,0.000000e+00,0.008662,-0.653419,50.672294,-45.455374,0.00000,0.000000,0.389277,0.000000
Group_8,0.008687,-0.703263,0.093431,0.097851,51.791219,-44.201218,3.039697,3.370603,0.000000,0.000000e+00,...,0.414438,0.000000e+00,0.008690,-0.673322,51.689150,-47.627973,0.00000,0.000000,0.411703,0.000000
Group_9,0.008993,-0.702063,0.080922,0.100706,52.227852,-42.981827,2.564462,3.379021,0.000000,9.511752e-320,...,0.401239,0.000000e+00,0.008996,-0.672810,52.140397,-46.372537,0.00000,0.000000,0.398821,0.000000
